* The temporal order is scrambled in the test data making TS useless there.
* Still, just for learning / realism, we can still do it in the training data! 
* Let's add pseudo dates, and aggregate features on column subsets. Finally i'll run a model to predict the target!

In [1]:
import importlib
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from datetime import datetime
from scipy.special import logsumexp

from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import mean_squared_error, classification_report

In [2]:
train = pd.read_csv("caltech-cs155-2020/train.csv")
test = pd.read_csv("caltech-cs155-2020/test.csv")
df = pd.concat([train,test],sort=False)
print(df.shape)
print(df.columns)
df.tail()

(784239, 28)
Index(['id', 'last_price', 'mid', 'opened_position_qty ',
       'closed_position_qty', 'transacted_qty', 'd_open_interest', 'bid1',
       'bid2', 'bid3', 'bid4', 'bid5', 'ask1', 'ask2', 'ask3', 'ask4', 'ask5',
       'bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol', 'ask1vol',
       'ask2vol', 'ask3vol', 'ask4vol', 'ask5vol', 'y'],
      dtype='object')


,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
191854,784234,5150.6,5150.8,NaN,NaN,0.0,0,5150.6,5150.4,5150.2,...,5,3,4,2,1,3,1,3,2,NaN
191855,784235,5122.6,5122.7,1.0,2.0,3.0,-1,5122.6,5122.4,5122.2,...,1,5,1,20,16,17,4,1,16,NaN
191856,784236,5192.8,5192.3,NaN,NaN,0.0,0,5192.0,5191.8,5191.2,...,2,1,1,1,1,2,1,2,1,NaN
191857,784237,5152.0,5151.9,4.0,2.0,6.0,2,5151.8,5151.6,5151.4,...,5,6,8,37,7,1,2,1,1,NaN
191858,784238,5200.0,5200.3,2.0,0.0,2.0,2,5200.0,5199.0,5198.8,...,5,1,2,1,1,1,4,1,3,NaN


In [3]:
train.head()

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,bid2vol,bid3vol,bid4vol,bid5vol,ask1vol,ask2vol,ask3vol,ask4vol,ask5vol,y
0,0,3842.4,3842.6,NaN,NaN,103.0,0,3842.4,3842.0,3841.8,...,1,6,14,6,6,1,1,10,2,1
1,1,3842.8,3843.4,6.0,49.0,55.0,-43,3843.0,3842.8,3842.4,...,6,11,1,6,1,4,4,1,13,0
2,2,3844.0,3844.3,7.0,77.0,84.0,-69,3843.8,3843.6,3843.2,...,1,4,21,12,1,16,10,4,9,0
3,3,3843.8,3843.4,3.0,34.0,37.0,-30,3843.0,3842.8,3842.4,...,13,12,2,4,2,7,1,2,11,1
4,4,3843.2,3843.1,3.0,38.0,41.0,-35,3842.8,3842.4,3842.0,...,12,2,2,4,1,3,1,11,15,1


In [4]:
## y is binary.
display(train["y"].describe())

count    592380.000000
mean          0.356958
std           0.479103
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max           1.000000
Name: y, dtype: float64

In [5]:
bid_cols = ['bid1','bid2', 'bid3', 'bid4', 'bid5']
bid_vol_cols = ['bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol']
ask_cols = ['ask1', 'ask2', 'ask3', 'ask4', 'ask5',]
ask_vol_cols = ['ask1vol','ask2vol', 'ask3vol', 'ask4vol', 'ask5vol']

group_cols = {"bid_cols":bid_cols,"bid_vol_cols":bid_vol_cols,"ask_cols":ask_cols,"ask_vol_cols":ask_vol_cols}

* Additional features could include: rank, which bid number is the max/min, etc' 
* features between the aggregated features (e.g. max bid div max ask..)

In [6]:
for group in group_cols.keys():
    print(group)
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    df[f"{group}_min"] = df[group_cols[group]].min(axis=1)
    df[f"{group}_spread"] = df[f"{group}_max"].div(df[f"{group}_min"])
#     df[f"{group}_logsumexp"] = df[grou/_cols[group]].apply(logsumexp)
    
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    
df["last_price_div__mid"] = df["last_price"].div(df["mid"])

bid_cols
bid_vol_cols
ask_cols
ask_vol_cols


In [7]:
# df[bid_cols] = df[bid_cols] - df["mid"]
for col in bid_cols:
    df[f"{col}_sub"] = (df[col] - df["last_price"])/df["last_price"]
for col in ask_cols:
    df[f"{col}_sub"] = (df[col] - df["last_price"])/df["last_price"]
for i in range(5):
    df[f"{i}_imbal"] = (df[bid_vol_cols[i]]-df[ask_vol_cols[i]])/(df[bid_vol_cols[i]]+df[ask_vol_cols[i]])

df["mid_sub"] = (df["mid"] - df["last_price"])/df["last_price"]

df["interest_imbal"] = df["d_open_interest"]/df["transacted_qty"]
df["opened_rat"] = df["opened_position_qty "]/df["transacted_qty"]
df["closed_rat"] = df["closed_position_qty"]/df["transacted_qty"]


# df[ask_cols] = df[ask_cols] - df["mid"]

In [8]:
print(df.columns)
df.head()

Index(['id', 'last_price', 'mid', 'opened_position_qty ',
       'closed_position_qty', 'transacted_qty', 'd_open_interest', 'bid1',
       'bid2', 'bid3', 'bid4', 'bid5', 'ask1', 'ask2', 'ask3', 'ask4', 'ask5',
       'bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol', 'ask1vol',
       'ask2vol', 'ask3vol', 'ask4vol', 'ask5vol', 'y', 'bid_cols_max',
       'bid_cols_min', 'bid_cols_spread', 'bid_vol_cols_max',
       'bid_vol_cols_min', 'bid_vol_cols_spread', 'ask_cols_max',
       'ask_cols_min', 'ask_cols_spread', 'ask_vol_cols_max',
       'ask_vol_cols_min', 'ask_vol_cols_spread', 'last_price_div__mid',
       'bid1_sub', 'bid2_sub', 'bid3_sub', 'bid4_sub', 'bid5_sub', 'ask1_sub',
       'ask2_sub', 'ask3_sub', 'ask4_sub', 'ask5_sub', '0_imbal', '1_imbal',
       '2_imbal', '3_imbal', '4_imbal', 'mid_sub', 'interest_imbal',
       'opened_rat', 'closed_rat'],
      dtype='object')


,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1,bid2,bid3,...,ask5_sub,0_imbal,1_imbal,2_imbal,3_imbal,4_imbal,mid_sub,interest_imbal,opened_rat,closed_rat
0,0,3842.4,3842.6,NaN,NaN,103.0,0,3842.4,3842.0,3841.8,...,0.000416,0.142857,0.000000,0.714286,0.166667,0.500000,0.000052,0.000000,NaN,NaN
1,1,3842.8,3843.4,6.0,49.0,55.0,-43,3843.0,3842.8,3842.4,...,0.000520,0.750000,0.200000,0.466667,0.000000,-0.368421,0.000156,-0.781818,0.109091,0.890909
2,2,3844.0,3844.3,7.0,77.0,84.0,-69,3843.8,3843.6,3843.2,...,0.000468,0.500000,-0.882353,-0.428571,0.680000,0.142857,0.000078,-0.821429,0.083333,0.916667
3,3,3843.8,3843.4,3.0,34.0,37.0,-30,3843.0,3842.8,3842.4,...,0.000260,0.666667,0.300000,0.846154,0.000000,-0.466667,-0.000104,-0.810811,0.081081,0.918919
4,4,3843.2,3843.1,3.0,38.0,41.0,-35,3842.8,3842.4,3842.0,...,0.000468,0.866667,0.600000,0.333333,-0.692308,-0.578947,-0.000026,-0.853659,0.073171,0.926829


In [9]:
df = df.drop(columns=bid_cols)
df = df.drop(columns=ask_cols)

In [10]:
df.head()

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1vol,bid2vol,bid3vol,...,ask5_sub,0_imbal,1_imbal,2_imbal,3_imbal,4_imbal,mid_sub,interest_imbal,opened_rat,closed_rat
0,0,3842.4,3842.6,NaN,NaN,103.0,0,8,1,6,...,0.000416,0.142857,0.000000,0.714286,0.166667,0.500000,0.000052,0.000000,NaN,NaN
1,1,3842.8,3843.4,6.0,49.0,55.0,-43,7,6,11,...,0.000520,0.750000,0.200000,0.466667,0.000000,-0.368421,0.000156,-0.781818,0.109091,0.890909
2,2,3844.0,3844.3,7.0,77.0,84.0,-69,3,1,4,...,0.000468,0.500000,-0.882353,-0.428571,0.680000,0.142857,0.000078,-0.821429,0.083333,0.916667
3,3,3843.8,3843.4,3.0,34.0,37.0,-30,10,13,12,...,0.000260,0.666667,0.300000,0.846154,0.000000,-0.466667,-0.000104,-0.810811,0.081081,0.918919
4,4,3843.2,3843.1,3.0,38.0,41.0,-35,14,12,2,...,0.000468,0.866667,0.600000,0.333333,-0.692308,-0.578947,-0.000026,-0.853659,0.073171,0.926829


In [11]:
df["date"] = pd.to_datetime("1.1.2019")
df["date"] = df["date"] + pd.to_timedelta(df["id"]/2,unit="s") # 500 ms per row

df["date"].describe()

count                         784239
unique                        784239
top       2019-01-02 14:21:54.500000
freq                               1
first            2019-01-01 00:00:00
last             2019-01-05 12:55:19
Name: date, dtype: object

# Split back into train and test, and build model

# Define functions

In [12]:
def get_normalized_X_and_train_y(df, col):
    train_opq = df.loc[~df[col].isna()]
    print(f"train shape {train.shape[0]}")
    test_opq = df.loc[df[col].isna()]
    print(f"test shape {test.shape[0]}")
    print(train_opq.shape, test_opq.shape)

    X_opq = train_opq.drop(["opened_position_qty ", "closed_position_qty","interest_imbal", "opened_rat", "closed_rat", "d_open_interest","id","date","y"],axis=1)
    from scipy import stats
    normalized_X_opq = pd.DataFrame(stats.zscore(X_opq, axis=0))
    normalized_X_opq.columns = [X_opq.columns]
    y_opq = train_opq[col]

    print(normalized_X_opq.shape, y_opq.shape)


    test_X_opq = test_opq.drop(["opened_position_qty ", "closed_position_qty","interest_imbal", "opened_rat", "closed_rat", "d_open_interest", "id","date","y"],axis=1)

    normalized_test_X_opq = pd.DataFrame(stats.zscore(test_X_opq, axis=0))
    normalized_test_X_opq.columns = [test_X_opq.columns]
    normalized_test_X_opq.index = test_X_opq.index
    print(normalized_test_X_opq.shape)
#     print(normalized_test_X_opq.columns)
    return normalized_X_opq, y_opq, normalized_test_X_opq, train_opq, test_opq

In [13]:
def get_df_with_preds_for_nans(df, col):
    from sklearn import linear_model
    clf = linear_model.SGDClassifier(max_iter=10, tol=1e-3)
    clf.fit(normalized_X_opq, y_opq)

    pred_opq = clf.predict(normalized_test_X_opq)
    test_opq[col] = pred_opq
    df = pd.concat([train_opq,test_opq],sort=False)
    df = df.sort_values('id')
    return df

In [14]:
df

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1vol,bid2vol,bid3vol,...,0_imbal,1_imbal,2_imbal,3_imbal,4_imbal,mid_sub,interest_imbal,opened_rat,closed_rat,date
0,0,3842.4,3842.6,NaN,NaN,103.0,0,8,1,6,...,0.142857,0.000000,0.714286,0.166667,0.500000,0.000052,0.000000,NaN,NaN,2019-01-01 00:00:00.000
1,1,3842.8,3843.4,6.0,49.0,55.0,-43,7,6,11,...,0.750000,0.200000,0.466667,0.000000,-0.368421,0.000156,-0.781818,0.109091,0.890909,2019-01-01 00:00:00.500
2,2,3844.0,3844.3,7.0,77.0,84.0,-69,3,1,4,...,0.500000,-0.882353,-0.428571,0.680000,0.142857,0.000078,-0.821429,0.083333,0.916667,2019-01-01 00:00:01.000
3,3,3843.8,3843.4,3.0,34.0,37.0,-30,10,13,12,...,0.666667,0.300000,0.846154,0.000000,-0.466667,-0.000104,-0.810811,0.081081,0.918919,2019-01-01 00:00:01.500
4,4,3843.2,3843.1,3.0,38.0,41.0,-35,14,12,2,...,0.866667,0.600000,0.333333,-0.692308,-0.578947,-0.000026,-0.853659,0.073171,0.926829,2019-01-01 00:00:02.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191854,784234,5150.6,5150.8,NaN,NaN,0.0,0,1,5,3,...,0.000000,0.250000,0.500000,0.142857,0.000000,0.000039,NaN,NaN,NaN,2019-01-05 12:55:17.000
191855,784235,5122.6,5122.7,1.0,2.0,3.0,-1,3,1,5,...,-0.684211,-0.888889,0.111111,0.000000,0.111111,0.000020,-0.333333,0.333333,0.666667,2019-01-05 12:55:17.500
191856,784236,5192.8,5192.3,NaN,NaN,0.0,0,1,2,1,...,0.000000,0.000000,0.000000,-0.333333,0.000000,-0.000096,NaN,NaN,NaN,2019-01-05 12:55:18.000
191857,784237,5152.0,5151.9,4.0,2.0,6.0,2,4,5,6,...,-0.272727,0.666667,0.500000,0.777778,0.947368,-0.000019,0.333333,0.666667,0.333333,2019-01-05 12:55:18.500


In [15]:
normalized_X_opq, y_opq, normalized_test_X_opq, train_opq, test_opq = get_normalized_X_and_train_y(df, "opened_position_qty ")

train shape 592380
test shape 191859
(558123, 51) (226116, 51)
(558123, 42) (558123,)
(226116, 42)


In [16]:
df = get_df_with_preds_for_nans(df, "opened_position_qty ")

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [17]:
df

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1vol,bid2vol,bid3vol,...,0_imbal,1_imbal,2_imbal,3_imbal,4_imbal,mid_sub,interest_imbal,opened_rat,closed_rat,date
0,0,3842.4,3842.6,14.0,NaN,103.0,0,8,1,6,...,0.142857,0.000000,0.714286,0.166667,0.500000,0.000052,0.000000,NaN,NaN,2019-01-01 00:00:00.000
1,1,3842.8,3843.4,6.0,49.0,55.0,-43,7,6,11,...,0.750000,0.200000,0.466667,0.000000,-0.368421,0.000156,-0.781818,0.109091,0.890909,2019-01-01 00:00:00.500
2,2,3844.0,3844.3,7.0,77.0,84.0,-69,3,1,4,...,0.500000,-0.882353,-0.428571,0.680000,0.142857,0.000078,-0.821429,0.083333,0.916667,2019-01-01 00:00:01.000
3,3,3843.8,3843.4,3.0,34.0,37.0,-30,10,13,12,...,0.666667,0.300000,0.846154,0.000000,-0.466667,-0.000104,-0.810811,0.081081,0.918919,2019-01-01 00:00:01.500
4,4,3843.2,3843.1,3.0,38.0,41.0,-35,14,12,2,...,0.866667,0.600000,0.333333,-0.692308,-0.578947,-0.000026,-0.853659,0.073171,0.926829,2019-01-01 00:00:02.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191854,784234,5150.6,5150.8,0.0,NaN,0.0,0,1,5,3,...,0.000000,0.250000,0.500000,0.142857,0.000000,0.000039,NaN,NaN,NaN,2019-01-05 12:55:17.000
191855,784235,5122.6,5122.7,1.0,2.0,3.0,-1,3,1,5,...,-0.684211,-0.888889,0.111111,0.000000,0.111111,0.000020,-0.333333,0.333333,0.666667,2019-01-05 12:55:17.500
191856,784236,5192.8,5192.3,7.0,NaN,0.0,0,1,2,1,...,0.000000,0.000000,0.000000,-0.333333,0.000000,-0.000096,NaN,NaN,NaN,2019-01-05 12:55:18.000
191857,784237,5152.0,5151.9,4.0,2.0,6.0,2,4,5,6,...,-0.272727,0.666667,0.500000,0.777778,0.947368,-0.000019,0.333333,0.666667,0.333333,2019-01-05 12:55:18.500


In [18]:
normalized_X_opq, y_opq, normalized_test_X_opq, train_opq, test_opq = get_normalized_X_and_train_y(df, "closed_position_qty")

train shape 592380
test shape 191859
(558123, 51) (226116, 51)
(558123, 42) (558123,)
(226116, 42)


In [19]:
df = get_df_with_preds_for_nans(df, "closed_position_qty")

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:603: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [20]:
df["interest_imbal"] = df["d_open_interest"]/df["transacted_qty"]
df["opened_rat"] = df["opened_position_qty "]/df["transacted_qty"]
df["closed_rat"] = df["closed_position_qty"]/df["transacted_qty"]

In [21]:
df

,id,last_price,mid,opened_position_qty,closed_position_qty,transacted_qty,d_open_interest,bid1vol,bid2vol,bid3vol,...,0_imbal,1_imbal,2_imbal,3_imbal,4_imbal,mid_sub,interest_imbal,opened_rat,closed_rat,date
0,0,3842.4,3842.6,14.0,14.0,103.0,0,8,1,6,...,0.142857,0.000000,0.714286,0.166667,0.500000,0.000052,0.000000,0.135922,0.135922,2019-01-01 00:00:00.000
1,1,3842.8,3843.4,6.0,49.0,55.0,-43,7,6,11,...,0.750000,0.200000,0.466667,0.000000,-0.368421,0.000156,-0.781818,0.109091,0.890909,2019-01-01 00:00:00.500
2,2,3844.0,3844.3,7.0,77.0,84.0,-69,3,1,4,...,0.500000,-0.882353,-0.428571,0.680000,0.142857,0.000078,-0.821429,0.083333,0.916667,2019-01-01 00:00:01.000
3,3,3843.8,3843.4,3.0,34.0,37.0,-30,10,13,12,...,0.666667,0.300000,0.846154,0.000000,-0.466667,-0.000104,-0.810811,0.081081,0.918919,2019-01-01 00:00:01.500
4,4,3843.2,3843.1,3.0,38.0,41.0,-35,14,12,2,...,0.866667,0.600000,0.333333,-0.692308,-0.578947,-0.000026,-0.853659,0.073171,0.926829,2019-01-01 00:00:02.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191854,784234,5150.6,5150.8,0.0,1.0,0.0,0,1,5,3,...,0.000000,0.250000,0.500000,0.142857,0.000000,0.000039,NaN,NaN,inf,2019-01-05 12:55:17.000
191855,784235,5122.6,5122.7,1.0,2.0,3.0,-1,3,1,5,...,-0.684211,-0.888889,0.111111,0.000000,0.111111,0.000020,-0.333333,0.333333,0.666667,2019-01-05 12:55:17.500
191856,784236,5192.8,5192.3,7.0,1.0,0.0,0,1,2,1,...,0.000000,0.000000,0.000000,-0.333333,0.000000,-0.000096,NaN,inf,inf,2019-01-05 12:55:18.000
191857,784237,5152.0,5151.9,4.0,2.0,6.0,2,4,5,6,...,-0.272727,0.666667,0.500000,0.777778,0.947368,-0.000019,0.333333,0.666667,0.333333,2019-01-05 12:55:18.500


In [22]:
df = df.replace([np.inf, -np.inf], 1)

In [23]:
train = df.loc[~df.y.isna()]
print(f"train shape {train.shape[0]}")
test = df.loc[df.y.isna()]
print(f"test shape {test.shape[0]}")

train shape 592380
test shape 191859


In [24]:
# train_no_nan = train.fillna(train.mean())
# test_no_nan = test.fillna(test.mean())
train_no_nan = train.fillna(-1)
test_no_nan = test.fillna(-1)

In [25]:
X = train_no_nan.drop(["id","date","y"],axis=1)
y = train_no_nan["y"]

In [26]:
y

0         1.0
1         0.0
2         0.0
3         1.0
4         1.0
         ... 
592375    1.0
592376    0.0
592377    0.0
592378    0.0
592379    0.0
Name: y, Length: 592380, dtype: float64

In [27]:
X.shape

(592380, 48)

In [35]:
import first_neural_network, second_neural_network, ensembleorder5, third_neural_network
importlib.reload(first_neural_network)
importlib.reload(second_neural_network)
importlib.reload(ensembleorder5)
importlib.reload(third_neural_network)

model = ensembleorder5.EnsembleOrder()

model.model1.batch_s = 64
model.model2.batch_s = 64
model.model4.batch_s = 64

model.model1.num_epochs = 15
model.model2.num_epochs = 15
model.model4.num_epochs = 15

model.model3.model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [36]:
import get_average_val_err_for_ensemble5

importlib.reload(get_average_val_err_for_ensemble5)

print(get_average_val_err_for_ensemble5.get_val_err(3, train_no_nan, model))

592380
Fold  1  of  3  test indices: [     0      1      2 ... 197457 197458 197459]
len(val_index_list):  197460
Train Epoch: 1  Loss: 0.6286
Train Epoch: 2  Loss: 0.7277
Train Epoch: 3  Loss: 0.4958
Train Epoch: 4  Loss: 0.5572
Train Epoch: 5  Loss: 0.5647
Train Epoch: 6  Loss: 0.5635
Train Epoch: 7  Loss: 0.5211
Train Epoch: 8  Loss: 0.5524
Train Epoch: 9  Loss: 0.5956
Train Epoch: 10  Loss: 0.5771
Train Epoch: 11  Loss: 0.6101
Train Epoch: 12  Loss: 0.5474
Train Epoch: 13  Loss: 0.4930
Train Epoch: 14  Loss: 0.5862
Train Epoch: 15  Loss: 0.5637
x train tensor:  tensor([[-2.1965, -2.1984, -0.6014,  ...,  0.4036, -0.2820,  0.7466],
        [-2.1965, -2.1965, -0.1355,  ...,  1.7969,  1.2033, -2.0324],
        [-2.1928, -2.1947, -0.1355,  ...,  1.1002,  0.4607, -0.6429],
        ...,
        [ 1.7876,  1.8080, -0.1355,  ..., -0.9896,  1.2033,  0.7466],
        [ 1.7876,  1.8080, -0.1355,  ..., -0.9896,  1.2033,  0.7466],
        [ 1.7876,  1.8080, -0.1355,  ..., -0.9896,  1.2033,  0.74

/Users/sharon/OneDrive/Documents/FirstYear/Winter2020/ML/TeamAPlus/randomforestorder.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model.fit(X,y)


x train tensor:  tensor([[-2.1965, -2.1984, -0.6014,  ...,  0.4036, -0.2820,  0.7466],
        [-2.1965, -2.1965, -0.1355,  ...,  1.7969,  1.2033, -2.0324],
        [-2.1928, -2.1947, -0.1355,  ...,  1.1002,  0.4607, -0.6429],
        ...,
        [ 1.7876,  1.8080, -0.1355,  ..., -0.9896,  1.2033,  0.7466],
        [ 1.7876,  1.8080, -0.1355,  ..., -0.9896,  1.2033,  0.7466],
        [ 1.7876,  1.8080, -0.1355,  ..., -0.9896,  1.2033,  0.7466]])
y train tensor:  tensor([[1.],
        [1.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])
Train Epoch: 1  Loss: 0.5995
Train Epoch: 2  Loss: 0.5702
Train Epoch: 3  Loss: 0.6186
Train Epoch: 4  Loss: 0.6434
Train Epoch: 5  Loss: 0.6304
Train Epoch: 6  Loss: 0.5912
Train Epoch: 7  Loss: 0.5258
Train Epoch: 8  Loss: 0.5923
Train Epoch: 9  Loss: 0.5559
Train Epoch: 10  Loss: 0.6122
Train Epoch: 11  Loss: 0.6904
Train Epoch: 12  Loss: 0.5311
Train Epoch: 13  Loss: 0.5810
Train Epoch: 14  Loss: 0.6260
Train Epoch: 15  Loss: 

/Users/sharon/OneDrive/Documents/FirstYear/Winter2020/ML/TeamAPlus/randomforestorder.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model.fit(X,y)


x train tensor:  tensor([[-1.2484, -1.2462,  5.9272,  ...,  0.3807, -0.0742, -1.6301],
        [-1.2439, -1.2372,  2.1958,  ..., -0.7097, -0.1139,  0.4607],
        [-1.2304, -1.2270,  2.6623,  ..., -0.7650, -0.1521,  0.5320],
        ...,
        [ 1.7568,  1.7692, -0.1363,  ..., -1.0140,  1.2059,  0.7628],
        [ 1.7568,  1.7692, -0.1363,  ..., -1.0140,  1.2059,  0.7628],
        [ 1.7568,  1.7692, -0.1363,  ..., -1.0140,  1.2059,  0.7628]])
y train tensor:  tensor([[1.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]])
Train Epoch: 1  Loss: 0.6258
Train Epoch: 2  Loss: 0.5747
Train Epoch: 3  Loss: 0.6304
Train Epoch: 4  Loss: 0.6476
Train Epoch: 5  Loss: 0.5973
Train Epoch: 6  Loss: 0.6418
Train Epoch: 7  Loss: 0.6643
Train Epoch: 8  Loss: 0.6115
Train Epoch: 9  Loss: 0.5856
Train Epoch: 10  Loss: 0.6470
Train Epoch: 11  Loss: 0.5964
Train Epoch: 12  Loss: 0.4981
Train Epoch: 13  Loss: 0.5801
Train Epoch: 14  Loss: 0.5811
Train Epoch: 15  Loss: 

/Users/sharon/OneDrive/Documents/FirstYear/Winter2020/ML/TeamAPlus/randomforestorder.py:28: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.model.fit(X,y)


x train tensor:  tensor([[-1.1733, -1.1707,  6.2503,  ...,  0.4659, -0.0509, -1.7570],
        [-1.1680, -1.1600,  2.3390,  ..., -0.6431, -0.0906,  0.3978],
        [-1.1521, -1.1481,  2.8279,  ..., -0.6993, -0.1287,  0.4713],
        ...,
        [ 1.5198,  1.5252, -0.1056,  ...,  0.4659,  0.4877, -0.7179],
        [ 1.5198,  1.5225, -0.5945,  ...,  0.4659, -0.2520,  0.7091],
        [ 1.5172,  1.5199, -0.5945,  ...,  0.4659, -0.2520,  0.7091]])
y train tensor:  tensor([[1.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [1.]])
Train Epoch: 1  Loss: 0.6622
Train Epoch: 2  Loss: 0.4839
Train Epoch: 3  Loss: 0.5893
Train Epoch: 4  Loss: 0.5750
Train Epoch: 5  Loss: 0.6481
Train Epoch: 6  Loss: 0.6013
Train Epoch: 7  Loss: 0.6511
Train Epoch: 8  Loss: 0.5387
Train Epoch: 9  Loss: 0.5538
Train Epoch: 10  Loss: 0.5641
Train Epoch: 11  Loss: 0.8320
Train Epoch: 12  Loss: 0.5399
Train Epoch: 13  Loss: 0.5766
Train Epoch: 14  Loss: 0.6794
Train Epoch: 15  Loss: 

In [37]:
test["Predicted"] = model.predict(test_no_nan.drop(["id","date","y"],axis=1))
test[["id","Predicted"]].to_csv("submission_test_processed_ensemble5_bs64_ne15.csv",index=False)

(191859, 1)
(191859, 1)
(191859, 1)
(191859, 1)
(191859, 1)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
